In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from google.cloud import bigquery
from google.oauth2 import service_account

In [5]:
credentials = service_account.Credentials.from_service_account_file("C:/Users/Federico.Cremer/OneDrive - Havas/Desktop/Science/Claro Peru/credentials.json")
client = bigquery.Client(credentials = credentials, project = "claro-peru-trasabilidad")

In [ ]:
#print("Running:")

#list_datasets_1 = ["DV360", "GoogleAds","Facebook","LinkedIn","Paid_Media","Twitter"]

columns_dv360 = ["date", "data_source_name" ,"insertion_order", "campaign_name", "line_item", "clicks", "impressions", "conversions", "total_media_cost", "video_completions", "video_first_quartile_completes", "video_mid_points", "video_third_quartile_completes"]
columns_GoogleAds = ["date", "data_source_name", "campaign_name", "ad_group_name", "clicks", "impressions", "conversions", "cost", "video_quartile_100_views", "conversion_value", "video_views"]
columns_Facebook = ["date", "data_source_name", "campaign_name", "ad_group_name", "clicks", "impressions", "cost", "video_p_100_watched_actions", "video_p_25_watched_actions", "video_p_50_watched_actions", "video_p_75_watched_actions", "ad_name", "campaign_objective", "promoted_post_permalink_url", "promoted_post_permalink_ig_url", "action_link_click", "action_post_engagement", "landing_page_views", "conversion_value", "offsite_conversions", "offsite_conversions_fb_pixel_complete_registration", "offsite_conversions_fb_pixel_lead", "offsite_conversions_fb_pixel_purchase", "video_thruplay_watched_actions", "action_app_install", "action_type", "actions"]
columns_LinkedIn = ["date", "data_source_name","campaign_name", "clicks", "impressions", "conversions", "cost", "video_completions", "video_first_quartile_completions", "video_midpoint_completions", "video_third_quartile_completions", "campaign_objective", "total_engagements", "landing_page_clicks", "conversion_value", "video_views", "leads"]
columns_Paid_Media = ''
columns_Twitter = ["date", "data_source_name", "campaign_name", "line_item_name", "clicks", "impressions", "cost", "promoted_video_views_100", "promoted_video_views_25", "promoted_video_views_50", "promoted_video_views_75", "link_clicks"]

#### LINKEDIN / PAIDMEDIA / TWITTER

#### DV360 

#### GoogleAds

In [ ]:
df_GAds = pd.read_csv('df_GoogleAds.csv', usecols= columns_GoogleAds)

In [ ]:
df_GAds

In [ ]:
df_GAds.to_csv('df_GAds_Reducido.csv')

#### Facebook

In [ ]:
df_fb = pd.read_csv('df_facebook.csv' , usecols= columns_Facebook)

In [ ]:
df_fb.to_csv('df_fb_Reducido.csv')

#### Join/Merge de Dataframes reducidos a uno solo

In [ ]:
df_linkedIn_paidMedia_Twitter_Reducido = pd.read_csv('df_linkedIn_paidMedia_Twitter_Reducido.csv')
df_DV360 = pd.read_csv('df_DV360_Reducido.csv')

In [ ]:
df_total_final = pd.concat([df_linkedIn_paidMedia_Twitter_Reducido, df_GAds, df_DV360, df_fb])

In [ ]:
df_total_final

In [ ]:
df_total_final.to_csv('cuboMasterFinal.csv')

In [ ]:
df_total_final = pd.read_csv('cuboMasterFinal.csv')

In [6]:
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True)

In [7]:
with open('cuboMasterFinal.csv', "rb") as cuboMasterFinal: 
    
    upload_to_bigquery = client.load_table_from_file(cuboMasterFinal, destination= 'claro-peru-trasabilidad.cubo_test1.cubov1', job_config= job_config)
    upload_to_bigquery.result()

In [ ]:
print("DV360")
df_dv360 = client.merge_tables_from_dataset('DV360', columns=columns_dv360)

In [ ]:
df_dv360.to_csv('df_dv360.csv')

In [ ]:
print("GoogleAds")
df_GoogleAds = client.merge_tables_from_dataset('GoogleAds', columns=columns_GoogleAds)

In [ ]:
df_GoogleAds.to_csv('df_GoogleAds.csv')

In [ ]:
print("Facebook")
df_facebook = client.merge_tables_from_dataset('Facebook', columns=columns_Facebook)

In [ ]:
df_facebook.to_csv('df_facebook.csv')

In [ ]:
print("LinkedIn")
df_linkedin = client.merge_tables_from_dataset('LinkedIn', columns=columns_LinkedIn)

In [ ]:
df_linkedin.to_csv('df_linkedin.csv')

In [ ]:
print("Paid_Media")
df_paid_media = client.merge_tables_from_dataset('Paid_Media', columns=columns_Paid_Media)

In [ ]:
df_paid_media.to_csv('df_paid_media.csv')

In [ ]:
print("Twitter")
df_twitter = client.merge_tables_from_dataset('Twitter', columns=columns_Twitter)

In [ ]:
df_twitter.to_csv('df_twitter.csv')

In [ ]:
#list_datasets_2 = ["LinkedIn", "Paid_Media", "Twitter"]

#print("MERGE: LinkedIn", "Paid_Media", "Twitter")
#df_total_all_datasets_2 = client.merge_all_datasets(list_of_selected_datasets= list_datasets_2, columns=columns)

In [ ]:
#df_total_all_datasets_2

In [ ]:
#df_total_all_datasets_2.to_csv('linkedIn_paidMedia_Twitter.csv')

In [ ]:
# Esto no corre porque no trae todos los datos bien:
#dask_df = dd.read_csv('./*.csv', assume_missing= True)

#dask_df.compute().to_csv('./cubo_master.csv')

In [ ]:
#print("JOIN BOTH MERGES")
#df_total_final =  df_total_all_datasets_1.join(df_total_all_datasets_2)

In [ ]:
df_linkedIn_paidMedia_Twitter = pd.read_csv('cuboMaster2.csv')
df_total = pd.read_csv('df_dv360.csv')
df_GAds = pd.read_csv('df_GoogleAds.csv')
df_fb = pd.read_csv('df_facebook.csv')

In [ ]:
df_total = pd.concat([df_total, df_GAds])
df_total = pd.concat([df_total, df_fb])
df_total = pd.concat([df_total, df_linkedin])
df_total = pd.concat([df_total, df_paid_media])
df_total = pd.concat([df_total, df_twitter])

In [ ]:
df_total.head(5)

In [ ]:
print("TO CSV")
df_total.to_csv('cuboMaster.csv')

In [ ]:
print("UPLOAD TO BIGQUERY")
client.client.load_table_from_dataframe(df_total, destination= 'claro-peru-trasabilidad.cubo_test1.cubov1')

print('The program has finished.')